 Coleta de Dados


In [ ]:
!pip install pandas random

import pandas as pd
import random


idades = [random.randint(25, 60) for _ in range(1000)]
rendas = [random.randint(1500, 15000) for _ in range(1000)]  # Renda entre 1.5k e 15k
categorias = ['Perfume', 'Livro', 'Roupas', 'Eletrodoméstico', 'Tecnologia']
localizacoes = ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Porto Alegre', 'Curitiba', 'Brasilia']
estados_civis = ['Casada', 'Solteira', 'Divorciada', 'Viúva']

maes = pd.DataFrame({
    'ID': [i for i in range(1, 101)],
    'Idade': idades,
    'Renda': rendas,
    'localização': [random.choice(localizacoes) for _ in range(100)],
    'estado civil': [random.choice(estados_civis) for _ in range(100)],
    'filhos': [random.randint(0, 5) for _ in range(100)],
    'categoria': [random.choice(categorias) for _ in range(100)]
})


produtos = pd.DataFrame({
    'Produto': ['Perfume A', 'Livro B', 'Camiseta C', 'Liquidificador D', 'Smartphone E'],
    'Categoria': ['Perfume', 'Livro', 'Roupas', 'Eletrodoméstico', 'Tecnologia'],
    'Preço': [100, 50, 70, 150, 120],
})

print(maes.head())
print(produtos.head())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(10, 6))


sns.scatterplot(data=maes, x='Idade', y='Renda', hue='categoria', palette='Set1', s=100, alpha=0.7)


plt.title('Distribuição das Mães por Idade e Renda', fontsize=14)
plt.xlabel('Idade', fontsize=12)
plt.ylabel('Renda', fontsize=12)


plt.legend(title="Categoria", loc='upper right')
plt.grid(True)

plt.show()

categoria_count = maes['categoria'].value_counts()


plt.figure(figsize=(10, 6))
sns.barplot(x=categoria_count.index, y=categoria_count.values, palette='Set1')


plt.title('Distribuição de Categorias de Produto', fontsize=14)
plt.xlabel('Categoria de Produto', fontsize=12)
plt.ylabel('Quantidade de Mães', fontsize=12)

plt.grid(True)
plt.show()

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
maes[['Idade', 'Renda']] = scaler.fit_transform(maes[['Idade', 'Renda']])

print(maes.head())

   ID     Idade     Renda     localização estado civil  filhos   categoria
0   1 -0.258507 -1.710363  Belo Horizonte     Solteira       1       Livro
1   2 -1.546091  0.109975  Rio de Janeiro        Viúva       5      Roupas
2   3  1.623346  0.009698  Rio de Janeiro        Viúva       4  Tecnologia
3   4  0.830987  0.383323        Brasilia     Solteira       3     Perfume
4   5  0.038628 -0.478069    Porto Alegre        Viúva       5  Tecnologia


scrapi

In [ ]:
!pip install selenium pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random


def coletar_produtos_amazon(termo_busca, categoria_nome, min_itens=100, max_itens=200):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/113.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=options)

    produtos = []
    precos = []
    links = []
    categorias = []

    pagina = 1
    while len(produtos) < max_itens:
        url = f"https://www.amazon.com.br/s?k={termo_busca}&page={pagina}"
        driver.get(url)
        time.sleep(random.uniform(2, 4))

        resultados = driver.find_elements(By.CSS_SELECTOR, "div.s-main-slot div.s-result-item")

        if not resultados:
            break
        for r in resultados:
            try:
                titulo = r.find_element(By.TAG_NAME, "h2").text
                link = r.find_element(By.TAG_NAME, "a").get_attribute("href")

                try:
                    preco = r.find_element(By.CLASS_NAME, "a-price-whole").text
                    preco = int(preco.replace('.', '').replace(',', ''))
                except:
                    preco = None

                if preco:
                    produtos.append(titulo)
                    precos.append(preco)
                    links.append(link)
                    categorias.append(categoria_nome)

                if len(produtos) >= max_itens:
                    break
            except:
                continue
        pagina += 1

    driver.quit()

    df = pd.DataFrame({
        'Produto': produtos,
        'Categoria': categorias,
        'Preço': precos,
        'Link': links
    })

    if len(df) < min_itens:
        print(f"Atenção: Apenas {len(df)} itens coletados. A meta mínima era {min_itens}.")
    return df


categorias_busca = {
    "Perfume": "perfume feminino dia das mães",
    "Livro": "livro presente mãe",
    "Roupas": "roupa feminina presente mãe",
    "Eletrodoméstico": "liquidificador presente mãe",
    "Tecnologia": "smartphone presente mãe"
}


dfs = []
for categoria, termo in categorias_busca.items():
    df_categoria = coletar_produtos_amazon(
        termo_busca=termo,
        categoria_nome=categoria,
        min_itens=50,
        max_itens=200
    )
    dfs.append(df_categoria)


df_produtos = pd.concat(dfs, ignore_index=True)


df_produtos.to_csv("produtos_amazon_maes.csv", index=False, encoding='utf-8-sig')

print(df_produtos.head(10))

In [ ]:
from sklearn.neighbors import NearestNeighbors


knn = NearestNeighbors(n_neighbors=3)
knn.fit(maes[['Idade', 'Renda']])

perfil_mae = [[30, 5000]]
distancias, indices = knn.kneighbors(perfil_mae)


mothers_recommendations = maes.iloc[indices[0]]
print(mothers_recommendations)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotando todas as mães no gráfico
plt.figure(figsize=(10, 6))
sns.scatterplot(data=maes, x='Idade', y='Renda', hue='categoria', palette='Set1', s=100, alpha=0.7)

# Plotando o perfil da mãe fictícia
plt.scatter(perfil_mae[0][0], perfil_mae[0][1], color='red', s=200, label='Perfil Mãe Fictícia', edgecolor='black')

# Destacando as mães mais próximas
for i in range(len(mothers_recommendations)):
    plt.scatter(mothers_recommendations.iloc[i]['Idade'], mothers_recommendations.iloc[i]['Renda'],
                color='green', s=200, edgecolor='black', label=f'Mãe Próxima {i+1}' if i == 0 else "")

# Adicionando título, rótulos e legenda
plt.title('Mães Mais Próximas ao Perfil Fictício', fontsize=14)
plt.xlabel('Idade', fontsize=12)
plt.ylabel('Renda', fontsize=12)
plt.legend(title="Legenda", loc='upper left')

# Exibindo o gráfico
plt.grid(True)
plt.show()

In [ ]:
!pip install streamlit


import streamlit as st

# Título da aplicação
st.title('Recomendação de Produtos para o Dia das Mães')

# Entrada de dados do usuário
idade = st.slider('Idade da mãe', 18, 100, 30)
renda = st.slider('Renda mensal da mãe (R$)', 1500, 15000, 5000)

# Modelo de Recomendação
perfil_mae_input = [[idade, renda]]
distancias, indices = knn.kneighbors(perfil_mae_input)

# Exibindo as mães mais próximas
mothers_recommendations = maes.iloc[indices[0]]

# Exibindo os produtos recomendados
st.subheader("Mães semelhantes encontradas:")
st.write(mothers_recommendations)

st.subheader("Produtos recomendados:")
for categoria in mothers_recommendations['Idade']:
    if categoria == "Perfume":
        st.write(produtos[produtos['Categoria'] == "Perfume"])
    elif categoria == "Livro":
        st.write(produtos[produtos['Categoria'] == "Livro"])

!pip install streamlit pyngrok


2025-05-08 19:35:49.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.597 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.602 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 19:35:49.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar